In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models as torchvision_models
import timm
from sklearn.metrics import accuracy_score
import numpy as np
import tarfile

In [3]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score
import pickle

# Setting random seed for getting same accuracy matrix every time
seed = 45
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Function to print accuracy matrix
def print_accuracy_matrix(accuracy_matrix):
    n = len(accuracy_matrix)
    m = len(accuracy_matrix[0])
    for i in range(n):
        for j in range(i+1):
            print(f"{accuracy_matrix[i][j]:.2f}", end="\t")
        print()

# Function to load data
def load_data(file_path, device):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    features = data['features'] #storing features
    if 'targets' in data:
        targets = data['targets'] #storing targets
        targets = torch.tensor(targets, dtype=torch.long).to(device)
    else:
        targets = None # targets is None if data dont have targets
    features = torch.tensor(features, dtype=torch.float32).to(device)
    return features, targets


def class_weighted_clustering(features, prototypes, cov_matrices, num_classes=10):
    weights = []
    if len(prototypes) == 0:  # if there are no prototypes return features.shape[0] of ones
      return torch.ones(features.shape[0])
    for feature in features:
        distances = []
        for i in range(num_classes):
            diff = feature - prototypes[i]
            mah_dist = torch.sqrt(diff.T @ cov_matrices[i] @ diff)
            distances.append(mah_dist)
        # The weight is calculated based on the distance from the prototype/mean more the distance less the weight and viceversa
        weights.append(1 / (1 + torch.min(torch.stack(distances)) ** 2))
    return torch.tensor(weights)

# Initial LwP model training function using hard LWP model i.e each point belong to 1 class dont consider probability of belonging to other class
def train_lwp_hard_with_weights(features, targets, regularization=1e-2, prototype_smoothing=0.2, num_classes=10):
    num_samples = features.shape[0]
    prototypes = []   #storing prototype/mean of each class
    cov_matrices = []  #storing covariance of each class
    global_mean = features.mean(axis=0) # calculating global mean


    class_weights = class_weighted_clustering(features, prototypes=[], cov_matrices=[], num_classes=num_classes)

    for i in range(num_classes):
        # Add Gaussian noise to the features of each class
        class_features = features[targets == i]
        noisy_class_features = class_features + torch.normal(0, 0.1, size=class_features.shape).to(features.device)

        # smoothing the prototype with global mean
        prototype = (1 - prototype_smoothing) * noisy_class_features.mean(axis=0) + prototype_smoothing * global_mean
        prototypes.append(prototype)

        # covariance matrix regularizing
        cov_matrix = torch.cov(noisy_class_features.T) + regularization * torch.eye(class_features.shape[1]).to(features.device)
        cov_matrices.append(torch.inverse(cov_matrix))

    #calculate class weights after updation of the protypes and covariance matrices
    class_weights = class_weighted_clustering(features, prototypes=torch.stack(prototypes), cov_matrices=cov_matrices, num_classes=num_classes)

    return torch.stack(prototypes), cov_matrices, class_weights

# LwP model training function with soft clustering and progressive prototypes averaging i.e each points have probability of beloning to each class
def train_lwp_soft_with_weights(features, prev_prototypes, prev_cov_matrices, alpha=0.5, num_classes=10):
    device = features.device
    prototypes = []
    cov_matrices = []

    # Compute soft assignments z_nk for each data point and each class
    class_weights = class_weighted_clustering(features, prototypes=prev_prototypes, cov_matrices=prev_cov_matrices, num_classes=num_classes)

    class_weights = class_weights.to(device)

    # Compute soft assignments z_nk for each data point and each cluster
    soft_assignments = torch.zeros((features.shape[0], num_classes)).to(device)
    epsilon = 1e-12   # small value is taken to avoid division by zero error

    for k in range(num_classes):
        distances = torch.norm(features - prev_prototypes[k], dim=1) ** 2
        soft_assignments[:, k] = torch.exp(-distances)

    # add epsilon to soft_assignment_sum to avoid division by zero error
    soft_assignments_sum = soft_assignments.sum(axis=1, keepdim=True) + epsilon
    soft_assignments /= soft_assignments_sum

    for k in range(num_classes):
        # Update prototype (mean) with soft assignments
        weighted_sum = torch.sum(soft_assignments[:, k][:, None] * features, axis=0) * class_weights.view(-1, 1)
        total_weight = torch.sum(soft_assignments[:, k] * class_weights)

        # check whether total_weight > 0 or not
        if total_weight > 0:
            prototype = weighted_sum / total_weight
        else:
            # If total_weight is 0 use previous protype or use mean of features based on availability of prev_prototypes
            prototype = prev_prototypes[k] if prev_prototypes is not None else features.mean(axis=0)

        # update prototype based on given alpha value new prototype is weighted sum of current prototype and previous prototype
        prototype = alpha * prototype + (1 - alpha) * prev_prototypes[k]
        prototypes.append(prototype)

        # update covariance matrix with soft assignments
        cov_matrix = torch.zeros((features.shape[1], features.shape[1])).to(device)
        for n in range(features.shape[0]):
            diff = (features[n] - prototype).view(-1, 1)
            cov_matrix += soft_assignments[n, k] * (diff @ diff.T) * class_weights[n]
        if total_weight > 0:
            cov_matrix /= total_weight
        else:
            cov_matrix = torch.cov(features.T) + torch.eye(features.shape[1]).to(device)

        # update covariance matrix based on given alpha values new covariance matrix is weighted sum of current covariance matrix and previous covariance matrix
        cov_matrix = alpha * cov_matrix + (1 - alpha) * torch.inverse(prev_cov_matrices[k])
        cov_matrices.append(torch.inverse(cov_matrix + torch.eye(cov_matrix.shape[0]).to(device)))  #adding small value for stability purpose

    return torch.stack(prototypes), cov_matrices



#prediction function
def predict_lwp(features, prototypes, cov_matrices):
    predictions = []   # list to store predictions
    for feature in features:
        distances = []  # list to store distance with respect to each class
        for i in range(len(prototypes)):
            diff = feature - prototypes[i]
            mah_dist = torch.sqrt(diff.T @ cov_matrices[i] @ diff)   # calculating mahalanobis distance
            distances.append(mah_dist)
        predicted_class = torch.argmin(torch.tensor(distances))  #gives class which has less distance to the given feature
        predictions.append(predicted_class.item())
    return predictions # returning predictions

# Paths for datasets
dataset_paths = [fr'gdrive/My Drive/miniproject2/dataset/Extracted_Features_Part_One/extracted_features_{i}_data.tar.pkl' for i in range(1, 21)]
heldout_paths = [fr'gdrive/My Drive/miniproject2/dataset/Extracted_Features_Part_One/extracted_features_{i}_data.tar.pkl' for i in range(1, 21)]

models = []  # a list to store means and covariance matrices for each model
accuracy_matrix = np.zeros((20, 20))    # a matrix to store accuracy of each model vs heldout datasets


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training first model f1 on D1 using hard classification
features, targets = load_data(dataset_paths[0], device)
#get prototypes and covariance matrix for model 1 from train_lwp_hard function
prototypes, cov_matrices, _ = train_lwp_hard_with_weights(features, targets, regularization=1, prototype_smoothing=0.2)
models.append((prototypes, cov_matrices))  #appending means and covariance matrix to models list


#Updating models f2 to f20 using soft classification
for i in range(1, 20):
    features, targets = load_data(dataset_paths[i], device) #loading features
    # get prototypes and covariance matrices for model i+1 from train_lwp_soft function
    prototypes, cov_matrices = train_lwp_soft_with_weights(features, prev_prototypes=models[-1][0], prev_cov_matrices=models[-1][1], alpha=0.1)
    #appending means and covariance matrix to models list
    models.append((prototypes, cov_matrices))
    print(f"Update of model done using Dataset-{i+1}")

# Evaluation of models on held out datasets
for i, (prototypes, cov_matrices) in enumerate(models):  #taking  model f1 to f20 iteratively
    for j in range(i + 1):  #taking data set 1 to data set i for calculating goodness of i th  model on them
        heldout_features, heldout_targets = load_data(heldout_paths[j], device) #taking heldout features and targets
        heldout_predictions = predict_lwp(heldout_features, prototypes, cov_matrices) #get predictions from predict_lwp function
        accuracy = accuracy_score(heldout_targets.cpu().numpy(), heldout_predictions)
        accuracy_matrix[i, j] = accuracy * 100    # storing accuracy in terms of 0-100 range
    print(f"Evaluation of model {i+1} done!")

# Print accuracy matrix
print("Accuracy Matrix (Rows: Models f1 to f20, Columns: Held-out datasets D1 to D20):")
print_accuracy_matrix(accuracy_matrix)


<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:44: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  mah_dist = torch.sqrt(diff.T @ cov_matrices[i] @ diff)


Update of model done using Dataset-2
Update of model done using Dataset-3
Update of model done using Dataset-4
Update of model done using Dataset-5
Update of model done using Dataset-6
Update of model done using Dataset-7
Update of model done using Dataset-8
Update of model done using Dataset-9
Update of model done using Dataset-10
Update of model done using Dataset-11
Update of model done using Dataset-12
Update of model done using Dataset-13
Update of model done using Dataset-14
Update of model done using Dataset-15
Update of model done using Dataset-16
Update of model done using Dataset-17
Update of model done using Dataset-18
Update of model done using Dataset-19
Update of model done using Dataset-20
Evaluation of model 1 done!


<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)


Evaluation of model 2 done!


<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)


...

Evaluation of model 18 done!


<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor

Evaluation of model 19 done!


<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long).to(device)
<ipython-input-3-4f91a56f9da1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor

Evaluation of model 20 done!
Accuracy Matrix (Rows: Models f1 to f20, Columns: Held-out datasets D1 to D20):
99.96	
99.92	87.40	
99.92	87.44	88.04	
99.84	87.68	88.32	88.12	
99.72	87.56	88.24	88.40	87.72	
99.64	87.84	88.20	88.52	88.04	87.68	
99.44	88.12	88.04	88.48	87.96	87.76	87.16	
99.36	88.16	88.20	88.64	88.12	87.80	87.24	88.16	
99.12	88.40	87.96	88.52	88.00	87.72	87.40	88.40	87.56	
99.08	88.36	87.88	88.48	87.92	87.68	87.44	88.32	87.60	88.36	
98.88	88.44	88.04	88.84	88.00	87.88	87.56	88.32	87.40	88.64	74.72	
98.52	88.44	88.44	89.00	88.32	87.96	87.68	88.84	87.64	88.68	75.00	63.28	
98.24	88.40	88.56	88.92	88.44	88.00	87.84	88.76	87.56	88.72	75.00	63.20	77.72	
97.92	88.28	88.52	88.68	88.40	87.96	87.88	88.64	87.72	88.56	75.04	63.08	77.72	91.24	
97.64	88.16	88.72	88.76	88.40	87.88	88.00	88.68	87.48	88.68	74.88	63.04	77.44	91.12	86.72	
97.40	88.24	88.84	88.84	88.36	87.88	88.20	88.68	87.84	88.72	75.12	63.08	77.44	91.08	86.72	72.48	
97.12	88.32	88.76	88.88	88.40	87.84	88.12	88.64	87.68	88.60